In [7]:
# importing all the req libraries

import re
import os
import json
import shutil
import pandas as pd

In [13]:
# Making the file importing process as dynamic
schema_path = "C:\\Users\\User\\my_project\\Dsc\\Projects\\ML_Project_1_Wafer_fault_detection\\schema_training.json"
training_folder_path = "C:\\Users\\User\my_project\\Dsc\Projects\\ML_Project_1_Wafer_fault_detection\\Training_Batch_Files"

In [23]:
with open(schema_path, 'r') as d:
    d.read()
    
d

<_io.TextIOWrapper name='C:\\Users\\User\\my_project\\Dsc\\Projects\\ML_Project_1_Wafer_fault_detection\\schema_training.json' mode='r' encoding='cp1252'>

# Extracting values from training schema



In [24]:
def ValuesfromSchema():
    with open(schema_path, 'r') as f:
       #data = f.read()  converts all data to strings
        data = json.load(f)  # converting str to python dic format
    

    pattern = data['SampleFileName']
    LengthOfDateStampInFile = data['LengthOfDateStampInFile']
    LengthOfTimeStampInFile = data['LengthOfTimeStampInFile']
    NumberofColumns = data['NumberofColumns']
    ColName = data['ColName']
    return pattern, LengthOfDateStampInFile, LengthOfTimeStampInFile, NumberofColumns, ColName


In [37]:
pattern,LengthOfDateStampInFile,LengthOfTimeStampInFile,NumberofColumns,ColName = ValuesfromSchema()

In [38]:
pattern

'wafer_08012020_120000.csv'

In [39]:
LengthOfDateStampInFile

8

In [40]:
LengthOfTimeStampInFile

6

In [41]:
NumberofColumns

592

In [11]:
ColName

{'Wafer': 'varchar',
 'Sensor - 1': ' float',
 'Sensor - 2': ' float',
 'Sensor - 3': ' float',
 'Sensor - 4': ' float',
 'Sensor - 5': ' float',
 'Sensor - 6': ' float',
 'Sensor - 7': ' float',
 'Sensor - 8': ' float',
 'Sensor - 9': ' float',
 'Sensor - 10': ' float',
 'Sensor - 11': ' float',
 'Sensor - 12': ' float',
 'Sensor - 13': ' float',
 'Sensor - 14': ' float',
 'Sensor - 15': ' float',
 'Sensor - 16': ' float',
 'Sensor - 17': ' float',
 'Sensor - 18': ' float',
 'Sensor - 19': ' float',
 'Sensor - 20': ' float',
 'Sensor - 21': ' float',
 'Sensor - 22': ' float',
 'Sensor - 23': ' float',
 'Sensor - 24': ' float',
 'Sensor - 25': ' float',
 'Sensor - 26': ' float',
 'Sensor - 27': ' float',
 'Sensor - 28': ' float',
 'Sensor - 29': ' float',
 'Sensor - 30': ' float',
 'Sensor - 31': ' float',
 'Sensor - 32': ' float',
 'Sensor - 33': ' float',
 'Sensor - 34': ' float',
 'Sensor - 35': ' float',
 'Sensor - 36': ' float',
 'Sensor - 37': ' float',
 'Sensor - 38': ' float',


In [43]:

# defining a regex to validate the data

def Regex():
    regex = "['wafer']+['\_'']+[\d_]+[\d]+\.csv"
    return regex
    


In [44]:
regex = Regex()

In [45]:
mkdir good_data


In [46]:
mkdir bad_data

In [51]:
# Displaying files in training path for refernce
files = [f for f in os.listdir(training_folder_path)]
files[0:5]

['Wafer12_20012.csv',
 'Waferdefault_15010_130532.csv',
 'Wafer_07012020_000000.csv',
 'wafer_07012020_041011.csv',
 'Wafer_07012020_223345.csv']

In [52]:
def validationfilename(regex, LengthOfDateStampInFile, LengthOfTimeStampInFile):
    files = [f for f in os.listdir(training_folder_path)]
    for filename in files:
        if re.match(regex , filename):
            split_at_dot = re.split('.csv' , filename)
            split_at_dot = re.split('_' , split_at_dot[0])
            if len(split_at_dot[1]) == LengthOfDateStampInFile:
                if len(split_at_dot[2]) == LengthOfTimeStampInFile:
                    shutil.copy(f'{training_folder_path}\\{filename}' , 'good_data')
                else:
                    shutil.copy(f'{training_folder_path}\\{filename}' , 'bad_data')
            else:
                shutil.copy(f'{training_folder_path}\\{filename}' , 'bad_data')  
        else:	
            shutil.copy(f'{training_folder_path}\\{filename}' , 'bad_data')
    
    print('[Info] All the Files Are Processed')

In [53]:
validationf
ilename(regex, LengthOfDateStampInFile, LengthOfTimeStampInFile)

[Info] All the Files Are Processed


In [54]:
# validating column length in the files
def validate_columns(NumberofColumns):
    files = [f for f in os.listdir('good_data')]
    for filename in files:
        csv = pd.read_csv(f'good_data/{filename}')
        if csv.shape[1] == NumberofColumns:
            pass
        else:
            shutil.move(f'good_data/{filename}' , 'bad_data')
    
    print('[INFO]- Column checking Done')

In [55]:
validate_columns(NumberofColumns)

[INFO]- Column checking Done


In [56]:
# Validate Missing Value in Whole Column

def validate_missing_values_in_column():
    for file in os.listdir('good_data'):
        csv = pd.read_csv(f'good_data/{file}')
        count = 0
        for columns in csv:
            if (len(csv[columns]) - csv[columns].count()) == len(csv[columns]):
                count+=1
                shutil.move(f'good_data/{file}' , 'bad_data',)
                break
        if count == 0:
            csv.rename(columns={"Unnamed: 0":"Wafer"}, inplace = True)
            csv.to_csv(f'good_data/{file}', index = None , header = True)
            
    print('[info] Validation of column done')

In [57]:
validate_missing_values_in_column()

[info] Validation of column done


In [58]:
# combine all the file into 1 file


li = []
for file in os.listdir('good_data'):
    csv = pd.read_csv(f'good_data/{file}')
    li.append(csv)
        
combined_data = pd.concat(li, axis = 0, ignore_index=True)
combined_data.to_csv('main_data.csv')